# Visualización de datos en Altair

[Altair](https://altair-viz.github.io/index.html) is a declarative statistical visualization library for Python, based on Vega and Vega-Lite, and the source is available on GitHub.

With Altair, you can spend more time understanding your data and its meaning. Altair’s API is simple, friendly and consistent and built on top of the powerful Vega-Lite visualization grammar. This elegant simplicity produces beautiful and effective visualizations with a minimal amount of code. Altair offers a powerful and concise visualization grammar that enables you to build a wide range of statistical visualizations quickly.

Qué significa [**declarativo (declarative)**](https://en.wikipedia.org/wiki/Declarative_programming)
Declarativo  quiere decir que esta lenguaje se enfoca en **qué** queremos y no en cómo lo tendremos (no tenemos que escribir instrucciones detalladas de cómo hará el programa para graficar nuestros datos). 

Está basado en la gramática de los gráficos: 

A grammar of graphics is a tool that enables us to concisely describe the components of a graphic. Such a grammar allows us to move beyond named graphics (e.g., the “scatterplot”) and gain insight into the deep structure that underlies statistical graphics. [(Wickham, 2010)](https://byrneslab.net/classes/biol607/readings/wickham_layered-grammar.pdf).

Los elementos que componen al GoG son:
- Datos
- Coordenadas
- Escalas
- Estética
- Geometrías
- Estadísticas
- Facetas 


https://towardsdatascience.com/a-comprehensive-guide-to-the-grammar-of-graphics-for-effective-visualization-of-multi-dimensional-1f92b4ed4149




In [3]:
import pandas as pd
import altair as alt

In [1]:
#pip install altair

     |████████████████████████████████| 658 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 282 kB 20.1 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
Note: you may need to restart the kernel to use updated packages.


Vamos a trabajar con los datos del[reporte de felicidad 2023](https://www.kaggle.com/ajaypalsinghlo/world-happiness-report-2021/code). Altair asume que nuestra base de datos es _ordenada_("tidy")<sup>1</sup>.


<sup>1</sup>Las bases de datos _tidy_ son fáciles de manejar, modelar y visualizar, y tienen una estructura específica: cada variable es una columna, cada observación es una fila, y cada tipo de unidad observacional es una tabla. [(Wickham, 2014)](https://www.jstatsoft.org/article/view/v059i10)

In [5]:
happy_all = pd.read_csv("./data/world-happiness-report-2023.csv")
happy_23 = pd.read_csv("./data/world-happiness-report-2023-countries.csv")
happy_21 = pd.read_csv("./data/world-happiness-report-2021.csv")

In [11]:
#happy_23.head(20)

h_all_22 = happy_all[happy_all['year'] == 2022]



#happy_21[['Country name', "Regional indicator" ]]

In [23]:
happy = pd.merge(h_all_22,
                  happy_21[['Country name', "Regional indicator" ]],
                  on = "Country name",
                  #how  = "left",
                 #how  = "inner", de repente esto no es necesario.
                 indicator = True)

In [25]:
print(h_all_22.shape)
print(happy_21.shape)
print(happy.shape)




(114, 11)
(149, 20)
(111, 13)


## El objeto `Chart`

El objeto más importante de Altair es el `Chart`, que toma como único argumento un dataframe. 


In [7]:
#alt.Chart(happy)



Ya podemos empezar a decidir nuestro gráfico. Primero queremos decidir qué marcas (marks) geométricas queremos que tenga. Elegimos el atributo `mark_point()`. 
Así como `point`, existen otros tipos de _marks_
- point
- line
- bar
- area
- rect
- geoshape
- text
- circle
- square
- rule
- tick

In [27]:
#alt.Chart(happy).mark_point()

alt.Chart(happy).mark_point()

alt.Chart(...)

Aquí todavía no estamos dando una ubicación a los datos, así que está graficando todo en un mismo punto. Sin embargo, podemos especificar posiciones para nuestos puntos. Para ello usaremos el método `encode` para darle _encodings_ visuales a nuestos datos. En este caso, en el eje y colocaremos a los países de Latinoamérica.

In [29]:
happy_lac = happy[happy['Regional indicator'] == "Latin America and Caribbean"]

In [31]:
alt.Chart(happy_lac).mark_point().encode(
x = "Country name", 
)

alt.Chart(...)

El método del encoding crea un un mapeo de key-value entre varios encodings (x, y, color, forma, tamaño, etc) a las variables de nuestra base de datos, tan sólo usando sus nombres. Noten que Altair detecta el tipo de datos de un dataframe de pandas.  Aún tenemos varios puntos que se superponen, los separaremos agregando un encoding para x. 


In [35]:
alt.Chart(happy_lac).mark_point().encode(
x = "Country name", 
 y = "Life Ladder"
)

alt.Chart(...)

In [12]:
#happy_lac[['Life Ladder', 'Country name']]

Una forma más generalizada de realizar este mismo códig, es agregar `alt.X`  y `alt.Y`, lo cual permitirá agregar más parámetros, y por tanto, más personalización a nuestro gráfico. 

In [37]:
alt.Chart(happy_lac).mark_point().encode(
x = alt.X("Country name"), 
y = alt.Y("Life Ladder")
)

alt.Chart(...)

In [39]:
alt.Chart(happy_lac).mark_point().encode(
x = alt.X("Country name", sort = "y"), # ordenando el eje x por y. 
y = alt.Y("Life Ladder")
)

alt.Chart(...)

¿ Qué pasa si queremos un gráfico de barras?

In [47]:
alt.Chart(happy_lac).mark_tick().encode(
y = alt.Y("Life Ladder"),
x = alt.X("Country name", sort = "y")
)

alt.Chart(...)

A veces es necesario decirle explícitamente a Altair el tipo de datos de nuestras variables. 
Estos tipos pueden ser/se especifican de la siguiente forma:

- `'nombre_variable:N'` Tipo de datos Nominal (datos categóricos no ordenados),

- `'nombre_variable:O'` Tipo de datos Ordinal (datos categóricos ordenados),

- `'nombre_variable:Q'` Tipo de datos cuantitativo (datos numéricos) y

- `'nombre_variable:T'` Variables de tiempo como fechas o años (casi no hemos visto esto)

Volviendo al primer ejemplo:

In [55]:
alt.Chart(happy_lac).mark_point().encode(
    y = alt.Y("Life Ladder:Q"),
    x = alt.X("Country name:N")
    )

alt.Chart(...)

En el siguiente ejemplo haremos un scatterplot (gráfico de dispersión) de la relación entre felicidad y PBI:

In [57]:
alt.Chart(happy).mark_point().encode(
    y = alt.Y("Log GDP per capita:Q"), # pbi per capita
    x = alt.X("Life Ladder:Q"), # felicidad
    )

alt.Chart(...)

Podemos agregar un encoding adicional: El color. En este caso, color significará la región de pertenencia. 

In [59]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = "Regional indicator:N"
    )
# Notar que  las regiones son variables nominales, por ello elige diferentes colores como encoding. 

alt.Chart(...)

¿Qué pasa si coloreamos por una variable cuantitativa?

In [19]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = 'Healthy life expectancy at birth'
    )

alt.Chart(...)

Agregando interactividad básica:

In [62]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = "Regional indicator"
    ).interactive()


alt.Chart(...)

Qué pasaría si queremos agregar el _tooltip_ 

In [64]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = "Regional indicator",
    tooltip = ["Country name", "Life Ladder"]
    ).interactive()

alt.Chart(...)

Qué pasa si queremos agregar un gráfico por región:

In [66]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = alt.Color("Regional indicator"),
    column = alt.Column("Regional indicator")
    )

alt.Chart(...)

In [68]:
alt.Chart(happy).mark_point(filled = True).encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = alt.Color("Regional indicator", 
            legend=alt.Legend(orient='bottom', titleOrient='left')
                     ),
    column = alt.Column("Regional indicator")
    ).properties(width=150, height=150)

alt.Chart(...)

Qué pasa si queremos agregar un encoding más respecto al tamaño de los puntos:

In [71]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = alt.Color("Regional indicator"),
    size = alt.Size("Generosity"),
    tooltip = ["Country name", "Life Ladder"]

    ).interactive()

alt.Chart(...)

In [73]:
happy_peru = happy_all[happy_all['Country name'] == 'Peru']

In [75]:
happy_peru

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1534,Peru,2006,4.811,8.979,0.875,66.460,0.668,-0.073,0.895,0.675,0.420
1535,Peru,2007,5.214,9.054,0.756,66.720,0.638,-0.080,0.931,0.730,0.361
1536,Peru,2008,5.129,9.134,0.777,66.980,0.638,-0.069,0.896,0.701,0.354
1537,Peru,2009,5.519,9.138,0.799,67.240,0.638,-0.082,0.880,0.758,0.320
1538,Peru,2010,5.613,9.210,0.812,67.500,0.757,-0.063,0.881,0.744,0.330
1539,Peru,2011,5.892,9.263,0.756,67.760,0.773,-0.126,0.824,0.742,0.331
1540,Peru,2012,5.825,9.313,0.764,68.020,0.703,-0.082,0.867,0.705,0.398
1541,Peru,2013,5.783,9.361,0.797,68.280,0.703,-0.069,0.870,0.741,0.390
1542,Peru,2014,5.866,9.374,0.819,68.540,0.722,-0.139,0.878,0.743,0.319
1543,Peru,2015,5.577,9.394,0.798,68.800,0.802,-0.093,0.884,0.744,0.378


In [77]:
elem1 = alt.Chart(happy_peru
                 ).mark_line().encode(
x = alt.X('year'),
y = alt.Y('Life Ladder:Q')
)

elem1

alt.Chart(...)

In [79]:
elem2 = alt.Chart(happy_peru).mark_area(color='blue', opacity = 0.5).encode(
x = alt.X('year'),
y = alt.Y('Life Ladder:Q')
)

#

elem2

alt.Chart(...)

In [81]:
elem1 + elem2

alt.LayerChart(...)